In [ ]:
!pip install yfinance
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import yfinance as yf

     |████████████████████████████████| 5.5MB 11.1MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.55-py2.py3-none-any.whl size=22618 sha256=6eb2a9f02764440af01a1327dc8aa0e8f894c83459b2d6a425e876ec6656b514
  Stored in directory: /root/.cache/pip/wheels/04/98/cc/2702a4242d60bdc14f48b4557c427ded1fe92aedf257d4565c
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [ ]:
Stock = "RELIANCE.NS"
data = yf.download(Stock, start="2016-01-01", end="2020-10-31")
T3 = pd.DataFrame({"Close": data["Close"]})
T3['Year'] = T3.index.year 
T2 = pd.DataFrame({"cumpnl_long":['0'], "cumpnl_short":['0'], "cumpnl":['0'], "SMA":['0'],  "LMA":['0'], "Stock":['0'], "Year":['0']})

for z in range (2016, 2021, 1):
  T = T3.where(T3.Year == z)
  T = T.dropna()
  for x in range(1,35,2):
    for y in range(x,35,2):
      SMA=x
      LMA=y

      # Compute Moving averages of last 10 days and 30 days closing prices
      T['Short_average'] = T['Close'].rolling(window=SMA, min_periods=1, center=False).mean()
      T['Long_average'] = T['Close'].rolling(window=LMA, min_periods=1, center=False).mean()

      T['long_entry'] = T.Short_average > T.Long_average   
      T['long_exit'] = T.Short_average <= T.Long_average
      T['positions_long'] = np.nan  
      T.loc[T.long_entry,'positions_long'] = 1  
      T.loc[T.long_exit,'positions_long'] = 0  
      T.positions_long = T.positions_long.fillna(method='ffill')  

      T['short_entry'] = T.Short_average < T.Long_average   
      T['short_exit'] = T.Short_average >= T.Long_average
      T['positions_short'] = np.nan  
      T.loc[T.short_entry,'positions_short'] = -1  
      T.loc[T.short_exit,'positions_short'] = 0  
      T.positions_short = T.positions_short.fillna(method='ffill')  

      T['price_difference']= T.Close - T.Close.shift(1)
      T['pnllong'] = T.positions_long.shift(1) * T.price_difference
      T['pnlshort'] = T.positions_short.shift(1) * T.price_difference
      T['pnl'] = T['pnllong']+T['pnlshort'] 
      T['cumpnl_long'] = T.pnllong.cumsum()
      T['cumpnl_short'] = T.pnlshort.cumsum()
      T['cumpnl'] = T.pnl.cumsum()
      T1 = T[['cumpnl_short', 'cumpnl_long', 'cumpnl']].tail(1)
      T1['SMA'] = SMA
      T1['LMA'] = LMA
      T1['Stock'] = Stock
      T1['Year'] = z
      T2 = T2.append(T1)

print(T2)

[*********************100%***********************]  1 of 1 completed
                    cumpnl_long cumpnl_short   cumpnl  ... LMA        Stock  Year
0                             0            0        0  ...   0            0     0
2016-12-30 00:00:00           0            0        0  ...   1  RELIANCE.NS  2016
2016-12-30 00:00:00    -55.6749     -99.2249   -154.9  ...   3  RELIANCE.NS  2016
2016-12-30 00:00:00    -9.02502      -52.575 -61.6001  ...   5  RELIANCE.NS  2016
2016-12-30 00:00:00     4.82504      -38.725 -33.8999  ...   7  RELIANCE.NS  2016
...                         ...          ...      ...  ...  ..          ...   ...
2020-10-30 00:00:00      619.95      18.1998   638.15  ...  31  RELIANCE.NS  2020
2020-10-30 00:00:00       370.8      -230.95   139.85  ...  33  RELIANCE.NS  2020
2020-10-30 00:00:00           0            0        0  ...  31  RELIANCE.NS  2020
2020-10-30 00:00:00      399.45       -180.9   218.55  ...  33  RELIANCE.NS  2020
2020-10-30 00:00:00          

In [ ]:
Pivot_Table1 = pd.pivot_table(T2, values ='cumpnl', index =['SMA', 'LMA'], 
                         columns =['Year'], aggfunc = np.sum) 
print (Pivot_Table1)

Year        2016     2017     2018     2019     2020    0
SMA LMA                                                  
1   1          0        0        0        0        0  NaN
    3     -154.9  488.699    554.9  63.4506  127.339  NaN
    5   -61.6001   331.15    524.8  315.951  133.399  NaN
    7   -33.8999   285.55    -54.7  403.851  396.019  NaN
    9    -110.25   237.05 -6.49988  158.351  633.919  NaN
...          ...      ...      ...      ...      ...  ...
29  33  -95.3751  9.12488  57.5499      183   139.85  NaN
31  31         0        0        0        0        0  NaN
    33   -67.375    129.9    140.6   126.25   218.55  NaN
33  33         0        0        0        0        0  NaN
0   0        NaN      NaN      NaN      NaN      NaN    0

[154 rows x 6 columns]


In [ ]:
Pivot_Table1.to_csv("PV_T.csv", index=True, encoding='utf8')
from google.colab import files
files.download('PV_T.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>